#  Using the Formulas and Data I have now i can create a pipeline that inputs a simple table of distances and outputs all the Velocities I need 

In [1]:
#import modules
import numpy as np
import pandas as pd
from astroquery.gaia import Gaia
import matplotlib.pyplot as plt
from astropy.table import Column, Table
from astropy.io import ascii

import astropy.coordinates as coords
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.visualization import astropy_mpl_style

In [2]:
#import constants
RUWE= 1.4 #Mitch
R0 = 8.5 # kpc distance to the sun from galactic center
sun_curve = 220 #km/s fich 1988, brand 1993
### find these values
#Johnson 1986
ra_np = np.radians(192.25) # deg to radians right acsension of north pole
dec_np = np.radians(27.4) # deg to radians decliatoin of north pole
theta_o = np.radians(123) # deg to radiansradians
k  = 4.74 #km/s per mas/yr 

#solar values
# solar motion km/s 
# M. Carretero-Castrillo 2023 and Ried 2019
U_sun = 10.8
V_sun = 13.6
W_sun = 7.6

In [3]:
def make_query(list_ID, query):
    '''Make a query for gaia DR3 using a list of GAIA identifers
    input:
    list_ID  - list: gaia source IDs, make sure IDs are clean
    query - str: SQL query to use in GAIA
    return;
    Results- Astropy Table- query results from GAIA 
    
    '''
    #convert list to string map
    str_map = ','.join(map(str,list_ID))
    job = Gaia.launch_job(query)
    results =job.get_results()
    return results

In [4]:
def modify_parllax_add_distance(table):
    #offset Parallax
    p_offset = 0.021 # from GAIA EDR3
    table['parallax'] = table['parallax'] - p_offset # from GAIA EDR3
    #add parallax units
    table['parallax'].unit= u.mas
    #add the distance to the table
    distance = 1/table['parallax'] # kpc

    table.add_column(distance, name='distance') # distance from object to sun in kpc
    table['distance'].unit= u.kpc
    return table


In [5]:
def Galaxy_dist(table):
    '''find the distance an object is to the center of the galaxy based on law of cosine
    input:
    long (l)- galactic longitutde in degree
    lat(b)- galactic latitute in degrees 
    object_dist (d) - distance from the sun to the object in kpc
    return 
    R - distance from the object to the galactic centre in kpc
    '''
    long = table['l']
    lat = table['b']
    long_rad = np.radians(long) #convert to radians
    lat_rad = np.radians(lat)
    obj_dist = table['distance'] # kpc
    R_sqrd = R0**2 + (obj_dist**2 * np.cos(lat_rad)**2) - 2*R0*obj_dist*np.cos(long_rad)*np.cos(lat_rad)
    galactic_dist = np.sqrt(R_sqrd)
    table.add_column(galactic_dist, name = 'galactic distance')
    table['galactic distance'].unit = u.kpc
    return table

In [6]:
def rotation_curve(table):
    '''From brand 1993 and fich 1988
    Calculate the circular velocity of a star based on its galactocentric distance
    return in km/s'''
    #best fit constants for rotational curve  fit
    a1 = 1.00767
    a2 = 0.0394
    a3 = 0.00712
    # fit from Brand 1988
    gal_dist = table['galactic distance']
    theta  = a1*((gal_dist/R0)**a2) + a3
    theta = sun_curve*theta #km/s
    table.add_column(theta, name='circular velocity')
    table['circular velocity'].unit= u.km/u.s
    return table


In [7]:
def vlsr_model(table):
    '''Calculate the Velocity as Local Standard of Rest based on 
    1) Galactocentric distance
    2) Circular Velocity due to galactic rotation (see rotation_curve function)
    3) Galactic Latitude and Longitude
    
    Compare with a model from Brand 1993 which reduces solar motion
    input:
    table- astropy table
    plot- boolean'''
    long = table['l']
    lat = table['b']
    gal_dist = table['galactic distance']
    theta = table['circular velocity']
    
    long_rad = np.radians(long)
    lat_rad = np.radians(lat)
    V_lsr = (theta * (R0/gal_dist) - theta_o)*np.sin(long)*np.cos(lat)
    table.add_column(V_lsr, name='LSR velocity')
    table['LSR velocity'].unit= u.km/u.s
    return table

In [8]:
def make_sky(table):
    '''Create a galactic coordinate table from astropy table proper motion, ra,dec and distance to object'''
    ra = table['ra'] # deg
    dec= table['dec']#deg
    pmra= table['pmra'] #mas/yr
    pmdec = table['pmdec']
    distance = table['distance'] #kpc
    sky = SkyCoord(ra = ra* u.deg, dec =dec *u.deg, pm_ra_cosdec= pmra*u.mas/u.yr, pm_dec=pmdec*u.mas/u.yr, distance=distance*u.kpc, frame='icrs')
    galactic= sky.transform_to('galactic')
    pm_l = galactic.pm_l_cosb
    pm_b = galactic.pm_b
    table.add_column(pm_l, name='pm_l')
    table.add_column(pm_b, name='pm_b')
    return table

In [9]:
def transform_space_velocity(ra, dec):
    '''Johnson 1986 calculate space velocities
    Use a transformation Matrix to convert ra and dec corrdinates into space velocity components
                                                UVW

    Notes:
    Transform is correct for Johnson 1986 constants
    input:
    ra- right ascension of star in radians
    dec - declinatoin of star in radians

    Transform - see johnson 1986
    A- coordinate matrix
    Return the dot product of Transform and A

    
    '''
    
    T1= np.array([[np.cos(theta_o), np.sin(theta_o), 0.0],
                        [np.sin(theta_o), -np.cos(theta_o), 0.0],
                        [0.0,0.0,1.0]])
    T2 =  np.array([[-np.sin(dec_np), 0.0, np.cos(dec_np)],
                        [0.0,-1.0,0.0],
                        [np.cos(dec_np), 0.0, np.sin(dec_np)]])
    T3 = np.array([[np.cos(ra_np), np.sin(ra_np), 0.0],
                        [np.sin(ra_np), -np.cos(ra_np), 0.0],
                        [0.0,0.0,1.0]])
    # @ is a matrix operator
    #this is correct with Johnson 1986 values 
    Transform = T1 @ T2 @ T3 # transformation matrix
    A = np.array([[np.dot(np.cos(ra),np.cos(dec)), -np.sin(ra), np.dot(np.cos(ra),np.sin(dec))],
                 [np.dot(np.sin(ra), np.cos(dec)), np.cos(ra), -np.dot(np.sin(ra), np.sin(dec))],
                 [np.sin(dec), 0.0, np.cos(dec)]])
    #each star will have a unique matrix
    B = Transform @ A
    return B

In [10]:
def calculate_space_velocity(table):
    '''Caulate the space velocity of a star WRT the local standard of rest, subtracting solar motion'''

    UVW = []
    for row in table:
        ra = np.radians(row['ra'])
        dec = np.radians(row['dec'])
        
        pmra = row['pmra'] * 1e-3  # mas/yr to as/yr
        pmdec = row['pmdec'] * 1e-3
        
        prlx = row['parallax'] * 1e-3  # mas to as
        radial = row['radial_velocity']  # km/s
        B= transform_space_velocity(ra, dec) # calculate the transform tensor
    
        comp_array = np.array([radial, k*pmra/prlx, k*pmdec/prlx])
        UVW_val = np.dot(B,comp_array)
        #subtract solar motions
        UVW_val[0] = UVW_val[0] - U_sun
        UVW_val[1] = UVW_val[1] - V_sun
        UVW_val[2] = UVW_val[2] - W_sun
        UVW.append(UVW_val)
    UVW = np.array([UVW])
    space_U = UVW[:,0]*u.km/u.s
    space_V = UVW[:,1]*u.km/u.s
    space_W = UVW[:,2]*u.km/u.s
    table.add_columns([space_U,space_V,space_W],names=['U','V','W'])
    
    return table
    

In [11]:
def solar_proper_motion(table):
    '''Calculate the proper motion compoent due to solar motion in the galactic plane
    '''
    lat = table['b']
    long = table['l']
    dist = table['distance']
    #convert to radians
    long_rad = np.radians(long)
    lat_rad = np.radians(lat)
    
    Kr_mul_sol = U_sun*np.sin(long_rad) - V_sun*np.cos(lat_rad)
    mul_sol = (Kr_mul_sol/(k*dist)) * u.mas/u.yr # need to specifiy units to subtract from proper motion
    
    Kr_mub_sol = U_sun*np.cos(long_rad)*np.sin(lat_rad) + V_sun*np.sin(long_rad)*np.sin(lat_rad) - W_sun*np.cos(lat_rad)
    mub_sol = (Kr_mub_sol/(k*dist)) *u.mas/u.yr
    table.add_columns([mul_sol,mub_sol],names=['mu_l_sol','mu_b_sol'])
    return table

In [12]:
def flat_rotation_curve(table):
    '''Based off moffat 1998
    Model the proper motion in l and b due to the galactic rotation curve
    Uses flat rotation curve model good for 3 < R < 18 Kpc ( 2*R0)'''
    R = table['galactic distance']
    lat = table['b']
    long = table['l']
    dist = table['distance']
    omega_0 = sun_curve/R0 # km/s per kpc
    omega  =sun_curve/R
    long_rad = np.radians(long)
    lat_rad = np.radians(lat)
    #calculate proper motion due to galactic rotation
    K_mul_rot = ((R0/(dist*np.cos(lat_rad))) *(omega-omega_0)*np.cos(long_rad))-omega
    mul_rot = K_mul_rot/(k)
    #add units
    mul_rot = mul_rot*u.mas/u.yr
    #do the same for latitudal proper motion
    K_mub_rot = -(R0 /dist)*(omega-omega_0)*np.sin(lat_rad)*np.sin(long_rad)
    mub_rot = K_mub_rot/k*u.mas/u.yr
    #ADD TO TABLE
    table.add_column(mul_rot,name='pm_l_rot')
    table.add_column(mub_rot,name='pm_b_rot')
    return table

# all together

In [13]:
def lay_pipe(table,filename,filetype):
    
    #offset the parallax and add distance data
    table = modify_parllax_add_distance(table)
    #sky object proper motions in l and b
    table = make_sky(table)
    #galactocentric distance
    table = Galaxy_dist(table)
    #circular velocity 
    table= rotation_curve(table)
    #local standard of rest
    table = vlsr_model(table)

    #space velocities 
    table = calculate_space_velocity(table)
    #solar proper motion
    table = solar_proper_motion(table)
    #flat rortation curve
    table = flat_rotation_curve(table)
    #write the table
    table.write(filename, format=filetype,overwrite=True)
    return table

In [15]:
test_table = ascii.read('HMXB_with XRBCATS.csv',format='csv')
#IT WORKS
test_output = lay_pipe(test_table, filename='Test_XRBCATS.ecsv',filetype='ascii.ecsv')

/tmp/ipykernel_26516/4265837733.py:16: UserWarning: Warning: converting a masked element to nan.
  comp_array = np.array([radial, k*pmra/prlx, k*pmdec/prlx])


In [16]:
test_output

source_id,ra,ra_error,dec,dec_error,pmra,pmra_error,pmdec,pmdec_error,parallax,parallax_error,radial_velocity,radial_velocity_error,phot_g_mean_mag,l,b,phot_bp_mean_mag,phot_rp_mean_mag,bp_rp,distance,pm_l,pm_b,galactic distance,circular velocity,LSR velocity,U,V,W,mu_l_sol,mu_b_sol,pm_l_rot,pm_b_rot
,,,,,,,,,mas,,,,,,,,,,kpc,mas / yr,mas / yr,kpc,km / s,km / s,km / s,km / s,km / s,mas / yr,mas / yr,mas / yr,mas / yr
int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64[3],float64[3],float64[3],float64,float64,float64,float64
168450545792009600,58.84615062234396,0.02955361269414425,31.045836091857545,0.02079164795577526,-1.282117252826291,0.05307381600141525,-1.8690907560201113,0.030308086425065994,1.606402622661778,0.03745640814304352,--,--,6.261696815490723,163.0813533466715,-17.136203125538678,6.439853668212891,5.930059909820557,0.509793758392334,0.6225089438306689,0.2998739750764102,-2.2466420505515643,9.070779259737565,223.8221991051486,8.192164638234658,nan .. nan,nan .. nan,nan .. nan,-3.3393209990613464,-1.824765450972002,-0.41974177977474353,-0.40228464945484055
184497471323752064,80.64680481184236,0.025397874414920807,37.6759929364201,0.01957647316157818,1.3049449567151956,0.04082612693309784,-3.9994793622028038,0.02804887294769287,0.7003697567669315,0.030094077810645103,--,--,7.2317328453063965,170.05338802722687,0.7103172589013076,7.376429080963135,6.921237945556641,0.45519113540649414,1.4278172213149678,4.034945160732326,-1.1907703635148816,9.90931690511838,224.59780831431172,57.2264855741973,nan .. nan,nan .. nan,nan .. nan,-1.7337056244708182,-1.1380520028248469,-0.129828401297794,0.009899749618565766
252878401557369088,70.24720739300419,0.011905986815690994,44.53034418647852,0.008385959081351757,0.10064623612212906,0.016362275928258896,-1.186127754569379,0.013658520765602589,0.357823380551682,0.015042701736092567,--,--,10.40385627746582,159.8470955000755,-1.2700235829277626,10.832097053527832,9.79090690612793,1.0411901473999023,2.794674843377278,0.9560106139046501,-0.7092759837376441,11.164505890385186,225.64830133598005,18.368090032277234,nan .. nan,nan .. nan,nan .. nan,-0.7455235821626214,-0.564460059124236,-0.43538497578589014,-0.03026681088944409
276644757710014976,64.92556123476727,0.012604312971234322,55.999360679043434,0.008108320645987988,-0.4735607216045463,0.01827656850218773,-0.5096656032780096,0.013492867350578308,0.1890882458297216,0.014864281751215458,--,--,10.766115188598633,149.17651611413766,4.133059262254314,11.439369201660156,9.855047225952148,1.5843219757080078,5.288536025134652,0.022268318000563104,-0.6953581136542384,13.307086696895857,227.20363746243194,78.18249341931684,nan .. nan,nan .. nan,nan .. nan,-0.32036362164644555,-0.30902086729050926,-0.7582378694465203,0.11708059389221737
414196617287885312,18.996040698840275,0.010982572101056576,59.15394486765568,0.012779636308550835,-2.4625832668630094,0.014893881045281887,-0.546486466252775,0.01677328534424305,0.3181933399322079,0.018039019778370857,--,--,11.412055969238281,126.08000772759438,-3.5676859152271287,11.667165756225586,10.951165199279785,0.7160005569458008,3.142743340300753,-2.3992767526950365,-0.7787392811514604,10.653230860548826,225.23481952872547,-65.38317533338797,nan .. nan,nan .. nan,nan .. nan,-0.3252508138927292,-0.5285405247763757,-2.5954292913099066,-0.15012200845992857
426558460884582016,14.177450998208093,1.8335996866226196,60.71672280471204,2.3911945819854736,--,--,--,--,--,--,--,--,2.0645833015441895,123.57698659515256,-2.1484240190698634,2.7258431911468506,2.31510853767395,0.4107346534729004,--,0.0,0.0,--,--,--,nan .. nan,nan .. nan,nan .. nan,-4.592490868185271,-7.795510044747213,8.5,-8.5
427234969757165952,9.290132580203235,0.008413794450461864,61.36013319063004,0.009818237274885178,-1.7956235405295953,0.010947090573608875,-0.5251300483100196,0.0136089781299233